In [ ]:
import requests

from datetime import datetime
from wasmtime import Store, Module, Instance, WasiConfig

class FLS:

    def __init__(self, wasm_path) -> None:
        self.SESSION = requests.session()
        self.wasm = self.load_wasm(wasm_path) if wasm_path else self.get_wasm()

        self.proof = None
        self.fs_payload = None

        self.access_token  = None
        self.refresh_token = None
       
        self.get_proof()
        self.set_fs_payload()

    def export_wasm(self, wasm_bytes):
        store = Store()

        wasi_config = WasiConfig()
        wasi_config.inherit_stdout()
        wasi_config.inherit_stderr()

        module   = Module(store.engine, wasm_bytes)
        instance = Instance(store, module, [])

        return {
            "store": store,
            "cdx": instance.exports(store)["cdx"],
            "rdx": instance.exports(store)["rdx"],
            "bdx": instance.exports(store)["bdx"],
            "ndx": instance.exports(store)["ndx"],
            "mdx": instance.exports(store)["mdx"],
        }

    def get_wasm(self):
        headers = {
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " \
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
        }

        response = self.SESSION.get(
                url=r"https://nepalstock.com/assets/prod/css.wasm", 
                headers=headers,
                verify=False
            )
        
        if response.ok and response.content:
            return self.export_wasm(response.content)
        
    def load_wasm(self, path):
        with open(path, "rb") as f:
            return self.export_wasm(f.read())
        
    def salt_access_token(self, access_token, salt):
        store = self.wasm["store"]

        index_cdx = self.wasm["cdx"](store, salt[0], salt[1], salt[2], salt[3], salt[4])
        index_rdx = self.wasm["rdx"](store, salt[0], salt[1], salt[3], salt[2], salt[4])
        index_bdx = self.wasm["bdx"](store, salt[0], salt[1], salt[3], salt[2], salt[4])
        index_ndx = self.wasm["ndx"](store, salt[0], salt[1], salt[3], salt[2], salt[4])
        index_mdx = self.wasm["mdx"](store, salt[0], salt[1], salt[3], salt[2], salt[4])

        return \
            access_token[             :index_cdx] + \
            access_token[index_cdx + 1:index_rdx] + \
            access_token[index_rdx + 1:index_bdx] + \
            access_token[index_bdx + 1:index_ndx] + \
            access_token[index_ndx + 1:index_mdx] + \
            access_token[index_mdx + 1:         ]
    
    def salt_refresh_token(self, refresh_token, salt):
        store = self.wasm["store"]

        index_cdx = self.wasm["cdx"](store, salt[1], salt[0], salt[2], salt[4], salt[3])
        index_rdx = self.wasm["rdx"](store, salt[1], salt[0], salt[2], salt[3], salt[4])
        index_bdx = self.wasm["bdx"](store, salt[1], salt[0], salt[3], salt[2], salt[4])
        index_ndx = self.wasm["ndx"](store, salt[1], salt[0], salt[3], salt[2], salt[4])
        index_mdx = self.wasm["mdx"](store, salt[1], salt[0], salt[3], salt[2], salt[4])

        return \
            refresh_token[             :index_cdx] + \
            refresh_token[index_cdx + 1:index_rdx] + \
            refresh_token[index_rdx + 1:index_bdx] + \
            refresh_token[index_bdx + 1:index_ndx] + \
            refresh_token[index_ndx + 1:index_mdx] + \
            refresh_token[index_mdx + 1:         ]
    
    def get_proof(self):
        headers = {
            "authority": "www.nepalstock.com",
            "method": "GET",
            "path": "/api/authenticate/prove",
            "scheme": "https",
            "accept": "application/json, text/plain, */*",
            "accept-encoding": "gzip, deflate, br, zstd",
            "accept-language": "en-US,en;q=0.9",
            "content-type": "application/json",
            "dnt": "1",
            "origin": "https://www.nepalstock.com",
            "priority": "u=1, i",
            "referer": "https://www.nepalstock.com/floor-sheet",
            "sec-ch-ua": '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"Windows"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "sec-gpc": "1",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "\
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
        }

        response = self.SESSION.get(
                url=r"https://www.nepalstock.com/api/authenticate/prove", 
                headers=headers,
                verify=False
            )

        if response.ok:
            self.set_proof(proof=response.json())


    def set_proof(self, proof):
        self.proof = proof

        self.access_token = self.salt_access_token(
            access_token=self.proof["accessToken"],
            salt=[
                self.proof["salt1"], 
                self.proof["salt2"], 
                self.proof["salt3"], 
                self.proof["salt4"], 
                self.proof["salt5"]
            ]
        )

        self.refresh_token = self.salt_refresh_token(
            refresh_token=self.proof["refreshToken"],
            salt=[
                self.proof["salt1"], 
                self.proof["salt2"], 
                self.proof["salt3"], 
                self.proof["salt4"], 
                self.proof["salt5"]
            ]
        )
    
    def set_fs_payload(self):

        salt=[
            self.proof["salt1"], 
            self.proof["salt2"], 
            self.proof["salt3"], 
            self.proof["salt4"], 
            self.proof["salt5"]
        ]

        day = datetime.now().day
        dummy_id   = 3
        dummy_data = [
            147, 117, 239, 143, 157, 312, 161, 612, 512, 804, 411, 527, 170, 511, 421, 667, 764, 621, 301, 106, \
            133, 793, 411, 511, 312, 423, 344, 346, 653, 758, 342, 222, 236, 811, 711, 611, 122, 447, 128, 199, \
            183, 135, 489, 703, 800, 745, 152, 863, 134, 211, 142, 564, 375, 793, 212, 153, 138, 153, 648, 611, \
            151, 649, 318, 143, 117, 756, 119, 141, 717, 113, 112, 146, 162, 660, 693, 261, 362, 354, 251, 641, \
            157, 178, 631, 192, 734, 445, 192, 883, 187, 122, 591, 731, 852, 384, 565, 596, 451, 772, 624, 691]
        
        i = dummy_data[dummy_id] + dummy_id + 2 * day
        self.fs_payload = {"id":i + salt[1 if i % 10 < 4 else 3] * day - salt[(1 if i % 10 < 4 else 3) - 1] + 91}
        # self.fs_payload = fs_payload

    def refresh_proof(self):
        if not self.proof:
            return

        headers = {
            "authority": "www.nepalstock.com",
            "method": "POST",
            "path": "api/authenticate/refresh-token",
            "scheme": "https",
            "accept": "application/json, text/plain, */*",
            "accept-encoding": "gzip, deflate, br, zstd",
            "accept-language": "en-US,en;q=0.9",
            "authorization": "Salter " + self.access_token,
            "content-type": "application/json",
            "dnt": "1",
            "origin": "https://www.nepalstock.com",
            "priority": "u=1, i",
            "referer": "https://www.nepalstock.com/floor-sheet",
            "sec-ch-ua": '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"Windows"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "sec-gpc": "1",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "\
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
        }

        response = self.SESSION.post(
                url=r"https://www.nepalstock.com/api/authenticate/refresh-token", 
                headers=headers, 
                json={"refreshToken": self.refresh_token}, 
                verify=False
            )
        
        if response.ok:
            self.set_proof(proof=response.json())
    
    def get_floorsheet(self, page, size):

        url = f"https://www.nepalstock.com/api/nots/nepse-data/floorsheet?page={page}&size={size}&sort=contractId,desc"
        path = "/".join(url.split("/")[3:])

        for _ in range(10):
            
            headers = {
                "authority": "www.nepalstock.com",
                "method": "POST",
                "path": path,
                "scheme": "https",
                "accept": "application/json, text/plain, */*",
                "accept-encoding": "gzip, deflate, br, zstd",
                "accept-language": "en-US,en;q=0.7",
                "authorization": "Salter " + self.access_token,
                "content-type": "application/json",
                "dnt": "1",
                "origin": "https://www.nepalstock.com",
                "priority": "u=1, i",
                "referer": "https://www.nepalstock.com/floor-sheet",
                "sec-ch-ua": '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
                "sec-ch-ua-mobile": "?0",
                "sec-ch-ua-platform": '"Windows"',
                "sec-fetch-dest": "empty",
                "sec-fetch-mode": "cors",
                "sec-fetch-site": "same-origin",
                "sec-gpc": "1",
                "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
            }

            response = self.SESSION.post(url, headers=headers, json=self.fs_payload, verify=False)

            if response.status_code == 401:
                print("FS 401'd")
                print("AT", "Salter " + self.access_token)
                print("RT", "Salter " + self.refresh_token)

                self.refresh_proof()
                continue

            if response.ok:
                return response.json()

In [ ]:
import time
import warnings
import pandas as pd

from tqdm import tqdm
from datetime import date
from requests.packages.urllib3.exceptions import InsecureRequestWarning

warnings.filterwarnings("ignore", category=InsecureRequestWarning)

fls = FLS(wasm_path=None)

floorsheet_data = []

floorsheet_p1 = fls.get_floorsheet(page=0, size=500)
total_pages   = floorsheet_p1["floorsheets"]["totalPages"]

floorsheet_data.extend(floorsheet_p1["floorsheets"]["content"])

for page in tqdm(range(1, total_pages)):
    time.sleep(1)
    floorsheet_data.extend(fls.get_floorsheet(page=page, size=500)["floorsheets"]["content"])

today = date.today().strftime("%Y-%m-%d")
df = pd.DataFrame(floorsheet_data)
df.to_parquet(f'{today}.parquet')

In [ ]:
# import json

# floorsheet_data = []

# with open("./2024-08-28.json", "r") as file:
#     for page in json.load(file):
#         floorsheet_data.extend(page["floorsheets"]["content"])

# # today = date.today().strftime("%Y-%m-%d")
# df = pd.DataFrame(floorsheet_data)
# df.to_parquet(f'2024-08-28.parquet')

In [ ]:
# they change the dummy_id and random addition at end

# salt = [73177, 72569, 81194, 96136, 61933]

# day = datetime.now().day
# dummy_id   = 3
# dummy_data = [
#     147, 117, 239, 143, 157, 312, 161, 612, 512, 804, 411, 527, 170, 511, 421, 667, 764, 621, 301, 106, \
#     133, 793, 411, 511, 312, 423, 344, 346, 653, 758, 342, 222, 236, 811, 711, 611, 122, 447, 128, 199, \
#     183, 135, 489, 703, 800, 745, 152, 863, 134, 211, 142, 564, 375, 793, 212, 153, 138, 153, 648, 611, \
#     151, 649, 318, 143, 117, 756, 119, 141, 717, 113, 112, 146, 162, 660, 693, 261, 362, 354, 251, 641, \
#     157, 178, 631, 192, 734, 445, 192, 883, 187, 122, 591, 731, 852, 384, 565, 596, 451, 772, 624, 691]

# i = dummy_data[dummy_id] + dummy_id + 2 * day
# {"id":i + salt[1 if i % 10 < 4 else 3] * day - salt[(1 if i % 10 < 4 else 3) - 1] + 91}